In [ ]:
#CREATE AND SAVE ADJACENCY MATRIX FOR A FOLDER WITH MULTIPLE EXCEL FILES
import os
import glob
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Hyperparameters
channelDim = 14
sRate = 8
timeDuration = 45
nWindow = sRate * timeDuration

def conn_xcorr(data):
    '''
    Calculate the connectivity to construct graph adjacency matrix by cross-correlation.
    Keyword arguments:
        data -- Original data
    Return arguments:
        G -- Calculated connectivity
    '''
    channel_dim = channelDim
    G = np.zeros([channel_dim, channel_dim])

    trial_data = data
    col_g, row_g = np.meshgrid(range(channel_dim), range(channel_dim))
    for (idx1, idx2) in zip(row_g.reshape(-1), col_g.reshape(-1)):
        xcorr = np.correlate(trial_data[idx1], trial_data[idx2], 'full')
        G[idx1][idx2] = np.max(np.abs(xcorr))

    G = (G + np.transpose(G, (1, 0))) / 2
    return G

def create_adj_matrix(data_con):
    graph_values = data_con.reshape(-1)
    # Thresholding with 60 percentile
    g_val_thresh = np.percentile(graph_values, 60)
    graph_values[graph_values < g_val_thresh] = 0
    graph_values[graph_values >= g_val_thresh] = 1

    adj_matrix = graph_values.reshape(channelDim, channelDim)
    return adj_matrix


# Define directories
input_dir = '/content/drive/MyDrive/filtered_afraid'
output_dir = '/content/drive/MyDrive/Adjacency_Afraid'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get list of Excel files in the input directory
file_list = glob.glob(os.path.join(input_dir, '*.xlsx'))

# For graph plotting
keys = list(range(channelDim))
channel_name = [str(v + 1) for v in keys]  # Adjust labels to be 1-based
channel_label = dict(zip(keys, channel_name))

# Process each file
for file_path in file_list:
    print(f"Processing {file_path}...")

    # Data loading
    temp = pd.read_excel(file_path, header=None)
    data = np.array(temp).T

    # Calculating connectivity
    data_con = conn_xcorr(data)

    # Creating adjacency matrix
    graph = create_adj_matrix(data_con)
    graph=np.array(graph)
    df=pd.DataFrame(graph,index=None)
    output_file = os.path.join(output_dir, f"adjacency_{os.path.basename(file_path)}")
    df.to_excel(output_file, index=False, header=False) # Save the adjacency matrix to Excel
'''output_file = os.path.join(output_dir, f"adjacency_{file_path}")
    df.to_excel(output_file, index=False, header=False)'''

'''    # Generate graph file name and path
    base_name = os.path.basename(file_path).replace('.xlsx', '')
    graph_path = os.path.join(output_dir, f'{base_name}_graph.png')

    # Plot and save the graph
    show_graph_with_labels(graph, mylabels=channel_label, graph_path=graph_path)'''

print("All files processed!")



Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_D_Q4.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data1.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data10.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data11.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data12.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data13.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data14.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data15.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data16.xlsx...
Processing /content/drive/MyDrive/filtered_afraid/filtered_difference_normalized_data17.xlsx...
Processing /content/drive/MyDrive/filtered_

In [ ]:
import pandas as pd
import os

input_dir = '/content/drive/MyDrive/Adjacency_Afraid'
data_frames = []
output_dir = '/content/drive/MyDrive/degree_matrix'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Get list of Excel files in the input directory
file_list = glob.glob(os.path.join(input_dir, '*.xlsx'))

# For graph plotting
keys = list(range(channelDim))
channel_name = [str(v + 1) for v in keys]  # Adjust labels to be 1-based
channel_label = dict(zip(keys, channel_name))

for file in file_list:
    temp = pd.read_excel(file_path, header=None)
    data = np.array(temp)

    m, n = data.shape
    new_mat = np.zeros((m, n))
    for i in range(n):
      new_mat[i][i] = np.sum(data, axis = -1)[i]
    df = pd.DataFrame(new_mat, index = None)

    output_file = os.path.join(output_dir, f"degree_using_{os.path.basename(file_path)}")
    df.to_excel(output_file, index=False, header=False)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os

# Define the directory containing the Excel files
source_directory = '/content/drive/MyDrive/Adjacency_Afraid'

# List to hold data from all files
data_frames = []

# Iterate over each file in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):  # Ensure the file is an Excel file
        file_path = os.path.join(source_directory, filename)

        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_path,header=None)
        data = np.asarray(df)
        m, n = data.shape
        if m!=14 or n!=14:
          print("Danger!!")
        # Append the DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list row-wise
concatenated_df = pd.concat(data_frames, ignore_index=True)#axi                                                                                             s=0 by default for row-wise concatenation

# Define the path for the output Excel file
output_file_path = '/content/drive/MyDrive/afraid_concat.xlsx'

# Save the concatenated DataFrame to a new Excel file
concatenated_df.to_excel(output_file_path, index=False,header=False)

print("Completed concatenating all Excel files and saved the result to a single Excel file.")


Completed concatenating all Excel files and saved the result to a single Excel file.


In [ ]:
import pandas as pd
import os

# Define the directory containing the Excel files
source_directory = '/content/drive/MyDrive/Adjacency_Afraid'

# List to hold data from all files
data_frames = []

# Iterate over each file in the source directory
for filename in os.listdir(source_directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):  # Ensure the file is an Excel file
        file_path = os.path.join(source_directory, filename)

        # Read the Excel file into a DataFrame
        df = pd.read_excel(file_path,header=None)

        # Append the DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list row-wise
concatenated_df = pd.concat(data_frames, ignore_index=True)#axis=0 by default for row-wise concatenation

# Define the path for the output Excel file
output_file_path = '/content/drive/MyDrive/afraid_concat.xlsx'

# Save the concatenated DataFrame to a new Excel file
concatenated_df.to_excel(output_file_path, index=False,header=False)

print("Completed concatenating all Excel files and saved the result to a single Excel file.")

Completed concatenating all Excel files and saved the result to a single Excel file.


In [ ]:
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

# Example data
# 500x3 feature matrix
#features = np.random.rand(500, 3)
df_adj = pd.read_excel('/content/drive/MyDrive/adjacency_full_concat.xlsx', header = None)
adjacency_matrix = df_adj.to_numpy()

# 500x500 adjacency matrix
#adjacency_matrix = np.random.randint(0, 2, (500, 500))
df_features = pd.read_excel('/content/drive/MyDrive/FeatureVect_full_concat.xlsx', header= None)
features = df_features.to_numpy()

# Making the adjacency matrix symmetric
#adjacency_matrix = (adjacency_matrix + adjacency_matrix.T) / 2

# 500x500 degree matrix
df_degree = pd.read_excel('/content/drive/MyDrive/degree_inverse_concat.xlsx', header= None)
degree_matrix = df_degree.to_numpy()

# Labels (500x1)
labels = np.random.randint(0, 3, 500)

# Convert to torch tensors
features = torch.FloatTensor(features)
labels = torch.LongTensor(labels)

# Add self-loops to the adjacency matrix

adjacency_matrix = adjacency_matrix + np.eye(adjacency_matrix.shape[0],14,k=0)
adjacency_matrix=torch.FloatTensor(adjacency_matrix)
# Normalize the adjacency matrix
#degree_matrix_inv_sqrt = np.diag(1.0 / np.sqrt(adjacency_matrix.sum(axis=1)))
#np.shape(degree_matrix_inv_sqrt)
#np.shape(adjacency_matrix)
degree_matrix_inv_sqrt = torch.FloatTensor(degree_matrix  )
adjacency_normalized = degree_matrix_inv_sqrt @ adjacency_matrix.T @ degree_matrix_inv_sqrt
adjacency_normalized = torch.FloatTensor(adjacency_normalized)

class GCNLayer(nn.Module):
  def __init__(self, in_features, out_features):
    super(GCNLayer, self).__init__()
    self.linear = nn.Linear(in_features, out_features)

  def forward(self, x, adj):
    x = torch.mm(adj, x)
    x = self.linear(x)
    return F.relu(x)

class GCN(nn.Module):
  def __init__(self, n_features, n_hidden, n_classes):
    super(GCN, self).__init__()
    self.gcn1 = GCNLayer(n_features, n_hidden)
    self.gcn2 = GCNLayer(n_hidden, n_classes)

  def forward(self, x,adj):
    x = self.gcn1(x, torch.transpose(adj,0,1))
    x = self.gcn2(x, torch.transpose(adj,0,1))
    return F.log_softmax(x, dim=1)

# Parameters
n_features = features.shape[1]
n_hidden = 16
n_classes = 3

# Model
model = GCN(n_features, n_hidden, n_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training
epochs = 200
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  output = model(features, adjacency_normalized)
  loss = criterion(output, labels)
  loss.backward()
  optimizer.step()

if (epoch + 1) % 20 == 0:
  print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Testing

model.eval()
with torch.no_grad():
  output = model(features, adjacency_normalized)
  predicted = torch.max(output.data, 1)
  accuracy = (predicted == labels).sum().item() / labels.size(0)
  print(f'Accuracy: {accuracy * 100:.2f}%')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (14x4200 and 14x16)

In [ ]:
print(np.shape(degree_matrix))
print(np.shape(degree_matrix_inv_sqrt))
print(np.shape(adjacency_matrix))

(4200, 14)
torch.Size([4200, 4200])
torch.Size([4200, 14])
